In [1]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance
from xgboost import cv
import pywt

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import metrics
from scipy import stats

import time

os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [2]:
def normalization_1(data):
    
    data_mean = np.mean(data,0)   #(72,)
    data_std = np.std(data,0,ddof=1) 
    data_ = (data - data_mean)/data_std
    
    return data_, data_mean , data_std

In [3]:
def normalization_2(data):
    
    data_min = np.min(data, 0)
    data_max = np.max(data, 0) 
    data_ = (data - data_min) / (data_max - data_min + 1e-7)
    
    return data_, data_min , data_max

In [4]:
#one hot code
def to_categorical(y, num_classes=None):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [5]:
# one hot code
def one_hot ( labels , Label_class ):
    one_hot_label = np.array([[ int (i == int (labels[j])) for i in range (Label_class)] for j in range ( len (labels))])      
    return one_hot_label

In [6]:
def add_window_wavelet(x, time_step, level_):
    
    x_window = []

    for i in range(x.shape[0]):
        series = x[i]
        series_window = []
        for j in range(int(series.shape[0]-time_step)):
            dat = series[j: j+time_step]
            fre_msg_var = []
            for k in range(dat.shape[-1]): #var_num

                coeffs = pywt.wavedec(dat[:,k], 'db1', level = level_, mode='sym')

                fre_msg_ = []
                for i in coeffs: # num_level * 3(mean + std + kurtosis)
                    fre_msg_.append(np.mean(i, 0))
                    fre_msg_.append(np.std(i, 0, ddof=1))
                    fre_msg_.append(stats.kurtosis(i))
                fre_msg_var.append(fre_msg_)

            fre_msg_var = np.array(fre_msg_var)

            series_window.append(fre_msg_var.reshape([-1]))
        x_window.append(series_window)
        
    return np.array(x_window)

In [7]:
def label_add_window(x, time_step):
    
    x_window = []

    for i in range(x.shape[0]):
        series = x[i]
        series_window = []
        for j in range(int(series.shape[0]-time_step)):
            dat = series[j: j+time_step]
            series_window.append(dat[0])
        x_window.append(series_window)
    
    return np.array(x_window)

In [8]:
folder = 'TEP_shapelet_preprocess_TrainSet_AllFault_win432/'

if not os.path.isdir(folder):
    os.mkdir(folder)

# Training set

In [9]:
num_var = 52
time_step = 110
level = 9

In [10]:
filedir = "TEP_shapelet_preprocess_TrainSet_AllFault_win432-2"
filenames = []

for filename in os.listdir(filedir):
    filenames.append(os.path.join(filedir,filename))

time_start = time.time()

for i in range(len(filenames)):
    tep_shapelet = np.load(filenames[i]) #(num_shapelet, series_length, num_var) 
    tep_shapelet_wavelet = add_window_wavelet(tep_shapelet, time_step, level)  #[num_shapelet, (series_length - window_size), (num_var*(num_level+1)*3) ]
    tep_shapelet_wavelet = tep_shapelet_wavelet.reshape([-1, num_var*(level+1)*3 ])
    label = np.ones(len(tep_shapelet_wavelet))*i
    print(tep_shapelet_wavelet.shape)
    
    if i == 0: 
        feature_train = tep_shapelet_wavelet
        label_train = label
    else: 
        feature_train = np.concatenate((feature_train, tep_shapelet_wavelet), 0)
        label_train = np.concatenate((label_train, label), 0)
    
print(feature_train.shape)
print(label_train.shape)
time_end = time.time()
print('time cost',time_end-time_start,'s')

故障類別數:  20
['TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault00.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault01.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault02.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault03.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault04.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault05.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault06.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault07.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault08.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault09.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault_win432-2\\train_selected_tep_fault10.npy', 'TEP_shapelet_preprocess_TrainSet_AllFault

C:\Users\WIN10\anaconda3\envs\tf1.12\lib\site-packages\pywt\_multilevel.py:45: UserWarning: Level value of 9 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
C:\Users\WIN10\anaconda3\envs\tf1.12\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\WIN10\anaconda3\envs\tf1.12\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(333, 1560)
(350, 1560)
(348, 1560)
(354, 1560)
(359, 1560)
(366, 1560)
(342, 1560)
(342, 1560)
(351, 1560)
(342, 1560)
(327, 1560)
(335, 1560)
(364, 1560)
(367, 1560)
(346, 1560)
(342, 1560)
(331, 1560)
(352, 1560)
(353, 1560)
(357, 1560)
(6961, 1560)
(6961,)
time cost 787.6052718162537 s


In [11]:
train_x = feature_train
train_y = label_train

np.save(folder + 'train_x', train_x) 
np.save(folder + 'train_y', train_y) 

# Val set

In [12]:
filedir = "TEP_shapelet_preprocess_ValSet_AllFault_win180-2"
filenames = []

for filename in os.listdir(filedir):
    filenames.append(os.path.join(filedir,filename))

time_start = time.time()

for i in range(len(filenames)):
    tep_shapelet = np.load(filenames[i]) #(num_shapelet, series_length, num_var) 
    tep_shapelet_wavelet = add_window_wavelet(tep_shapelet, time_step, level)  #[num_shapelet, (series_length - window_size), (num_var*(num_level+1)*3) ]
    tep_shapelet_wavelet = tep_shapelet_wavelet.reshape([-1, num_var*(level+1)*3 ])
    label = np.ones(len(tep_shapelet_wavelet))*i
    print(tep_shapelet_wavelet.shape)
    
    if i == 0: 
        feature_val = tep_shapelet_wavelet
        label_val = label
    else: 
        feature_val = np.concatenate((feature_val, tep_shapelet_wavelet), 0)
        label_val = np.concatenate((label_val, label), 0)
    
print(feature_val.shape)
print(label_val.shape)
time_end = time.time()
print('time cost',time_end-time_start,'s')

故障類別數:  20
['TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault00.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault01.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault02.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault03.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault04.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault05.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault06.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault07.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault08.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault09.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault10.npy', 'TEP_shapelet_preprocess_ValSet_AllFault_win180-2\\val_selected_tep_fault11.npy', 'TEP

In [13]:
val_x = feature_val
val_y = label_val

np.save(folder + 'val_x', val_x) 
np.save(folder + 'val_y', val_y) 

# Test set

In [14]:
filedir = "TEP_shapelet_preprocess_TestSet_AllFault_win225-2"
filenames = []

for filename in os.listdir(filedir):
    filenames.append(os.path.join(filedir,filename))

time_start = time.time()

for i in range(len(filenames)):
    tep_shapelet = np.load(filenames[i]) #(num_shapelet, series_length, num_var) 
    tep_shapelet_wavelet = add_window_wavelet(tep_shapelet, time_step, level)  #[num_shapelet, (series_length - window_size), (num_var*(num_level+1)*3) ]
    tep_shapelet_wavelet = tep_shapelet_wavelet.reshape([-1, num_var*(level+1)*3 ])
    label = np.ones(len(tep_shapelet_wavelet))*i
    print(tep_shapelet_wavelet.shape)
    
    if i == 0: 
        feature_test = tep_shapelet_wavelet
        label_test = label
    else: 
        feature_test = np.concatenate((feature_test, tep_shapelet_wavelet), 0)
        label_test = np.concatenate((label_test, label), 0)
    
print(feature_test.shape)
print(label_test.shape)
time_end = time.time()
print('time cost',time_end-time_start,'s')

故障類別數:  20
['TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault00.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault01.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault02.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault03.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault04.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault05.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault06.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault07.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault08.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault09.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selected_tep_fault10.npy', 'TEP_shapelet_preprocess_TestSet_AllFault_win225-2\\test_selecte

In [15]:
test_fault_x = feature_test
test_fault_y = label_test

np.save(folder + 'test_fault_x', test_fault_x) 
np.save(folder + 'test_fault_y', test_fault_y) 

In [18]:
train_x, data_mean , data_std = normalization_1(train_x)
val_x = (val_x - data_mean)/data_std
test_fault_x = (test_fault_x - data_mean)/data_std

C:\Users\WIN10\anaconda3\envs\tf1.12\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
C:\Users\WIN10\anaconda3\envs\tf1.12\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\WIN10\anaconda3\envs\tf1.12\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [25]:
dtrain = xgb.DMatrix(train_x, train_y)
dval = xgb.DMatrix(val_x, val_y)
dtest_fault = xgb.DMatrix(test_fault_x, test_fault_y)

In [36]:
params = {
'objective': 'multi:softprob',
'num_class': len(filenames),
'seed': 0,
'gamma': 0,
'max_depth': 2, #10
# 'random_state': 0,
'subsample': 0.5,
'min_child_weight': 2,
'lambda': 2,
'grow_policy': 'lossguide',
'eta': 0.007,
'eval_metric': ['merror'],
}

model = xgb.train(params, dtrain, 
          num_boost_round = 4000, 
          verbose_eval = 20, 
          early_stopping_rounds = 200, 
          evals=[(dtrain, 'train') , (dval, 'valid'), (dtest_fault, 'test_fault')],
          )

print('training finish')

[0]	train-merror:0.02787	valid-merror:0.12319	test_fault-merror:0.19481
[20]	train-merror:0.00546	valid-merror:0.12360	test_fault-merror:0.19374
[40]	train-merror:0.00546	valid-merror:0.13082	test_fault-merror:0.19204
[60]	train-merror:0.00460	valid-merror:0.12640	test_fault-merror:0.17224
[80]	train-merror:0.00388	valid-merror:0.12400	test_fault-merror:0.16415
[100]	train-merror:0.00359	valid-merror:0.12520	test_fault-merror:0.14179
[120]	train-merror:0.00259	valid-merror:0.11758	test_fault-merror:0.13839
[140]	train-merror:0.00230	valid-merror:0.11998	test_fault-merror:0.13455
[160]	train-merror:0.00129	valid-merror:0.11838	test_fault-merror:0.13030
[180]	train-merror:0.00129	valid-merror:0.11838	test_fault-merror:0.12944
[200]	train-merror:0.00086	valid-merror:0.11677	test_fault-merror:0.12284
[220]	train-merror:0.00086	valid-merror:0.11718	test_fault-merror:0.11731
[240]	train-merror:0.00072	valid-merror:0.11758	test_fault-merror:0.11518
[260]	train-merror:0.00057	valid-merror:0.11

In [37]:
y_pred = model.predict(xgb.DMatrix(val_x))
yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro

predictions = [round(value) for value in yprob]

# evaluate predictions
accuracy = accuracy_score(val_y, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))
print('Recall: %.4f' % metrics.recall_score(val_y, predictions, average='macro'))
print('F1-score: %.4f' % metrics.f1_score(val_y, predictions, average='macro'))
print('Precesion: %.4f' % metrics.precision_score(val_y, predictions, average='macro'))
print("confused matrix:")
print(confusion_matrix(val_y, predictions))
df = pd.DataFrame(confusion_matrix(val_y, predictions))
df.to_csv('./TEP_xgboost_wavelet_shapelet_wavelet_val.csv')

Accuracy: 90.16854%
Recall: 0.8907
F1-score: 0.8718
Precesion: 0.8671
混淆矩阵:
[[103   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0  83   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0 124   0   0   0   0   0   4   0   0   0   0  33   0   0   0   0
    0   0]
 [  0   0   0 142   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0 110   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0 174   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0 105   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   6   0   0   0   0 122   3   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0 111   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0 119   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 135   0   0   0   0   0   0   0


C:\Users\WIN10\anaconda3\envs\tf1.12\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
y_pred = model.predict(xgb.DMatrix(test_fault_x))
yprob = np.argmax(y_pred, axis=1)  # return the index of the biggest pro

predictions = [round(value) for value in yprob]

# evaluate predictions
accuracy = accuracy_score(test_fault_y, predictions)
print("Accuracy: %.5f%%" % (accuracy * 100.0))
print('Recall: %.4f' % metrics.recall_score(test_fault_y, predictions, average='macro'))
print('F1-score: %.4f' % metrics.f1_score(test_fault_y, predictions, average='macro'))
print('Precesion: %.4f' % metrics.precision_score(test_fault_y, predictions, average='macro'))
print("confused matrix:")
print(confusion_matrix(test_fault_y, predictions))
df = pd.DataFrame(confusion_matrix(test_fault_y, predictions))
df.to_csv('./TEP_xgboost_wavelet_shapelet_wavelet_test.csv')

Accuracy: 91.80328%
Recall: 0.9301
F1-score: 0.9014
Precesion: 0.9271
混淆矩阵:
[[255   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0 178   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0 183   0   0   0   0   0   0   0   0   0   0  23   0   0   0   0
    0   1]
 [  0   0   4 347   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0 235   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0 247   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0 180   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0 136   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   6   0   0   0   0   0 140   0   0   0   0   0  11   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0 288   0   0   0   0   0   0   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0 143   0   0   0   0   0   0   0


In [39]:
classes=['1','2','3','4','5','6','7','8','10','11','12','13','14','15','16','17','18','19','20','21']


def calculae_lable_prediction(confMatrix):
    l=len(confMatrix)
    for i in range(l):
        label_total_sum = confMatrix.sum(axis=1)[i]
        label_correct_sum=confMatrix[i][i]
        prediction = round(100 * float(label_correct_sum) / float(label_total_sum), 2)
        print('precision rate:'+classes[i]+":"+str(prediction)+'%')
        
calculae_lable_prediction(confusion_matrix(test_fault_y, predictions))

精确率:1:100.0%
精确率:2:100.0%
精确率:3:88.41%
精确率:4:98.86%
精确率:5:100.0%
精确率:6:100.0%
精确率:7:100.0%
精确率:8:100.0%
精确率:10:89.17%
精确率:11:100.0%
精确率:12:100.0%
精确率:13:6.84%
精确率:14:100.0%
精确率:15:85.29%
精确率:16:91.93%
精确率:17:100.0%
精确率:18:100.0%
精确率:19:100.0%
精确率:20:99.69%
精确率:21:100.0%


In [40]:

def calculate_label_recall(confMatrix):
    l = len(confMatrix)
    for i in range(l):
        label_total_sum = confMatrix.sum(axis=0)[i]
        label_correct_sum = confMatrix[i][i]
        prediction = round(100 * float(label_correct_sum) / float(label_total_sum), 2)
        print('recall rate:'+classes[i] + ":" + str(prediction) + '%')

calculate_label_recall(confusion_matrix(test_fault_y, predictions))

召回率:1:100.0%
召回率:2:100.0%
召回率:3:83.56%
召回率:4:100.0%
召回率:5:100.0%
召回率:6:100.0%
召回率:7:100.0%
召回率:8:47.89%
召回率:10:53.85%
召回率:11:100.0%
召回率:12:100.0%
召回率:13:100.0%
召回率:14:100.0%
召回率:15:76.89%
召回率:16:94.91%
召回率:17:100.0%
召回率:18:97.6%
召回率:19:100.0%
召回率:20:100.0%
召回率:21:99.59%
